In [8]:
%load_ext sql
%sql sqlite:///geo.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [9]:
result = %sql select pcd, latitude, longitude from postcode

 * sqlite:///geo.db
Done.


In [10]:
df = result.DataFrame()

In [11]:
df

,pcd,latitude,longitude
0,AB1 0AA,57.101474,-2.242851
1,AB1 0AB,57.102554,-2.246308
2,AB1 0AD,57.100556,-2.248342
3,AB1 0AE,57.084444,-2.255708
4,AB1 0AF,57.096656,-2.258102
...,...,...,...
2673013,ZE3 9JW,59.873651,-1.305697
2673014,ZE3 9JX,59.875286,-1.307502
2673015,ZE3 9JY,59.891572,-1.313847
2673016,ZE3 9JZ,59.892392,-1.310899


In [18]:
df['outcode'] = df['pcd'].str.split(" ").str[0]
df

,pcd,latitude,longitude,outcode
0,AB1 0AA,57.101474,-2.242851,AB1
1,AB1 0AB,57.102554,-2.246308,AB1
2,AB1 0AD,57.100556,-2.248342,AB1
3,AB1 0AE,57.084444,-2.255708,AB1
4,AB1 0AF,57.096656,-2.258102,AB1
...,...,...,...,...
2673013,ZE3 9JW,59.873651,-1.305697,ZE3
2673014,ZE3 9JX,59.875286,-1.307502,ZE3
2673015,ZE3 9JY,59.891572,-1.313847,ZE3
2673016,ZE3 9JZ,59.892392,-1.310899,ZE3


In [55]:
import numpy as np
import pandas as pd

In [44]:
loc = df.copy()
loc['latitude'] = df['latitude'].astype(float)
loc['latitude'] = np.where((loc['latitude'] > 90) | (loc['latitude'] < 10), np.nan, loc['latitude'])

loc['longitude'] = df['longitude'].astype(float)
loc['longitude'] = np.where(loc['latitude'].isnull(), np.nan, loc['longitude'])

loc = loc.groupby('outcode').agg(lat_min=('latitude', min), lat_max=('latitude', max), 
                                 lon_min=('longitude', min), lon_max=('longitude', max))
loc['dlat'] = loc['lat_max']-loc['lat_min']
loc['dlon'] = loc['lon_max']-loc['lon_min']
loc

,lat_min,lat_max,lon_min,lon_max,dlat,dlon
outcode,,,,,,
AB1,57.047082,57.663858,-2.489732,-2.048890,0.616776,0.440842
AB10,57.111340,57.155419,-2.150614,-2.093992,0.044079,0.056622
AB11,57.111340,57.150972,-2.155799,-2.048708,0.039632,0.107091
AB12,57.046582,57.148707,-2.289517,-2.060668,0.102125,0.228849
AB13,57.098017,57.148547,-2.288824,-2.218620,0.050530,0.070204
...,...,...,...,...,...,...
YO91,53.974709,53.978235,-1.075422,-1.065358,0.003526,0.010064
YO95,53.926665,53.926665,-0.815105,-0.815105,0.000000,0.000000
ZE1,60.122890,60.187721,-1.292689,-1.134229,0.064831,0.158460


In [50]:
loc[~loc['lat_min'].isnull()].sort_values('dlon')

,lat_min,lat_max,lon_min,lon_max,dlat,dlon
outcode,,,,,,
BS0,51.343881,51.343881,-2.957023,-2.957023,0.000000,0.000000
PA63,56.381755,56.381755,-5.715838,-5.715838,0.000000,0.000000
BN95,50.823154,50.823154,-0.330024,-0.330024,0.000000,0.000000
EC3B,51.514030,51.514030,-0.081823,-0.081823,0.000000,0.000000
PA74,56.532966,56.532966,-6.229753,-6.229753,0.000000,0.000000
...,...,...,...,...,...,...
AB3,56.769336,57.518487,-3.721049,-2.145127,0.749151,1.575922
PR2,53.753373,53.811397,-2.774777,-1.117020,0.058024,1.657757
PA80,56.535031,57.043854,-7.653531,-5.545481,0.508823,2.108050


In [46]:
df[df['outcode'] == "BT93"].sort_values('longitude')

,pcd,latitude,longitude,outcode
363415,BT93 0AD,54.217002,-7.388845,BT93
363528,BT93 0QT,54.271558,-7.442036,BT93
364269,BT93 6EJ,54.189263,-7.531842,BT93
364286,BT93 6FF,54.188879,-7.541818,BT93
363414,BT93 0AB,54.551311,-7.549214,BT93
...,...,...,...,...
363918,BT93 3DJ,54.466172,-8.163139,BT93
363873,BT93 3AD,99.999999,0.000000,BT93
364451,BT93 7YA,99.999999,0.000000,BT93
363838,BT93 2BR,99.999999,0.000000,BT93


In [64]:
max_loc = pd.DataFrame(
    loc[['lat_max', 'lon_max']].max()
)
max_loc

,0
lat_max,60.800694
lon_max,1.762773


In [65]:
min_loc = pd.DataFrame(
    loc[['lat_min', 'lon_min']].min()
)
min_loc

,0
lat_min,49.891974
lon_min,-8.163139


In [66]:
df[df.pcd == 'SE1 7RU']

,pcd,latitude,longitude,outcode
2057718,SE1 7RU,51.498669,-0.109943,SE1


In [2]:
from geopy.distance import geodesic

In [8]:
geodesic((55.498669, -0.109943), (55.498668, -0.109942)).m

0.12801935758571975

In [9]:
geodesic((55.49866, -0.10994), (55.49865, -0.10995)).m

1.2801936863582988

In [10]:
geodesic((55.4986, -0.1099), (55.4985, -0.1098)).m

12.801944990617077

In [11]:
geodesic((55.498, -0.109), (55.497, -0.108)).m

128.02026323444758

In [3]:
geodesic((52.467060, -1.931671), (52.467, -1.9317)).m 

6.961389818005479